In [1]:
%env HF_HOME=/mnt/LLM
%env OMP_NUM_THREADS=16
%env CUDA_VISIBLE_DEVICES=2
%load_ext autoreload
%autoreload 2
import sys; sys.path.insert(0, "..")

env: HF_HOME=/mnt/LLM
env: OMP_NUM_THREADS=16
env: CUDA_VISIBLE_DEVICES=2


In [2]:
import datasets
import torch
from src.find_important_tokens_eagle import EaModelForAutoJudge
from lm_eval_utils import GSM8KParser, GSM8KEvaluator
from prompts import GSM8KPrompts, llama_assistant_turn_end

base_model_path = "meta-llama/Llama-3.1-8B-Instruct"
EAGLE_model_path = "yuhuili/EAGLE-LLaMA3.1-Instruct-8B"
window_size = 8

model = EaModelForAutoJudge.from_pretrained(
    base_model_path=base_model_path,
    ea_model_path=EAGLE_model_path,
    use_eagle3="eagle3" in EAGLE_model_path.lower(),
    torch_dtype="auto", #was: torch.bfloat16
    device_map="auto",
    low_cpu_mem_usage=True,    
    depth=window_size - 1,
    total_token=window_size,
    do_sample=False, top_p=None, top_k=1, temperature=None
)
device = next(model.parameters()).device
tokenizer = model.get_tokenizer()
model.eval();


LlamaForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
2025-07-30 18:37:01,649 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in 

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
test_data = datasets.load_dataset("openai/gsm8k", "main", split="test")
(len(test_data) - 1) // 3 + 1

440

In [4]:
import pandas as pd
trained_head = pd.read_pickle("./trained_head.pkl")

def classifier(hidden_states: torch.Tensor):
    hidden_states = hidden_states.cpu().reshape(1, -1).float().numpy()
    hidden_states = trained_head["scaler"].transform(hidden_states)
    p_important, = trained_head["model"].predict_proba(hidden_states)[:, 1]
    return p_important > 0.12

In [ ]:
import numpy as np
accepted_tokens_history = []
num_steps_history = []
accuracy_history = []
parser, evaluator = GSM8KParser(), GSM8KEvaluator()

for sample in test_data.select(range(440,880)):
    prompt_with_shots = GSM8KPrompts.prompt_with_0_shots
    prompt = prompt_with_shots + sample["question"] + "\n" + GSM8KPrompts.formatting_prompt + llama_assistant_turn_end
    batch_input_ids = tokenizer(prompt, return_tensors='pt')['input_ids'].to(device)
    output_dict = current_response = model.eagenerate_with_statistics(
        input_ids=batch_input_ids, max_new_tokens=1024, max_length=4096, classifier=classifier
    )
    response_str = tokenizer.decode(output_dict["input_ids"].flatten())

    accuracy_history.append(evaluator([response_str], [sample["answer"]]))
    num_steps_history.append(len(output_dict["step_statistics"]))
    accepted_tokens_history.extend([d["accepted_tokens"] for d in output_dict["step_statistics"]])
    print(f"Samples: {len(accuracy_history)}\t|",
          f"Running Acc: {np.mean(accuracy_history):.5f}",
          f"|\t Accepted: {np.mean(accepted_tokens_history)}")

/home/jheuristic/exp/speculation_for_reasoning/notebooks/../src/find_important_tokens_eagle.py:275: UserWarning: Model recognized as Llama 3.x
  warnings.warn("Model recognized as Llama 3.x")


Samples: 1	| Running Acc: 1.00000 |	 Accepted: 3.2435897435897436


/home/jheuristic/exp/speculation_for_reasoning/notebooks/../src/find_important_tokens_eagle.py:275: UserWarning: Model recognized as Llama 3.x
  warnings.warn("Model recognized as Llama 3.x")


Samples: 2	| Running Acc: 1.00000 |	 Accepted: 3.085714285714286


/home/jheuristic/exp/speculation_for_reasoning/notebooks/../src/find_important_tokens_eagle.py:275: UserWarning: Model recognized as Llama 3.x
  warnings.warn("Model recognized as Llama 3.x")


...

/home/jheuristic/exp/speculation_for_reasoning/notebooks/../src/find_important_tokens_eagle.py:275: UserWarning: Model recognized as Llama 3.x
  warnings.warn("Model recognized as Llama 3.x")


Samples: 297	| Running Acc: 0.82155 |	 Accepted: 3.219893448861308


/home/jheuristic/exp/speculation_for_reasoning/notebooks/../src/find_important_tokens_eagle.py:275: UserWarning: Model recognized as Llama 3.x
  warnings.warn("Model recognized as Llama 3.x")


Samples: 298	| Running Acc: 0.82215 |	 Accepted: 3.2195691517309823


/home/jheuristic/exp/speculation_for_reasoning/notebooks/../src/find_important_tokens_eagle.py:275: UserWarning: Model recognized as Llama 3.x
  warnings.warn("Model recognized as Llama 3.x")
